In [2]:
!git clone https://github.com/dcs-sastra/Kosaksi-Pasapugazh-and-experiments.git

Cloning into 'Kosaksi-Pasapugazh-and-experiments'...
remote: Enumerating objects: 582, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 582 (delta 30), reused 33 (delta 9), pack-reused 495
Receiving objects: 100% (582/582), 123.86 MiB | 11.78 MiB/s, done.
Resolving deltas: 100% (408/408), done.
Updating files: 100% (478/478), done.


In [3]:
import os
import pandas as pd
import glob
import IPython.display as ipd

# Define the base directory where your year folders are located
base_dir = "/content/Kosaksi-Pasapugazh-and-experiments/data/raw/Health - HMIS"

# Initialize an empty list to store individual dataframes
all_data = []

# List of months
months = ['April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'March']

# Function to get the total column for each month
def get_total_column(month, columns):
    return [col for col in columns if month in col and 'Total' in col][0]

# Loop through the years from 2017 to 2020
for year in range(2017, 2020):
    # Construct the file path
    file_path = os.path.join(base_dir, f"{year}-{year+1}", f"{year}-{year+1}", "All_India.csv")

    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file
        df = pd.read_csv(file_path, header=[0,1])

        # Flatten the multi-index columns
        df.columns = [' '.join(col).strip() for col in df.columns.values]

        # Find the appropriate column names
        state_col = [col for col in df.columns if 'Unnamed: 0' in col][0]
        med_code_col = [col for col in df.columns if 'Unnamed: 1' in col][0]
        med_desc_col = [col for col in df.columns if 'Unnamed: 2' in col][0]
        stock_details_col = [col for col in df.columns if 'Unnamed: 3' in col][0]

        # Try to find the total column, if it exists
        total_col = [col for col in df.columns if 'Total Total' in col]
        total_col = total_col[0] if total_col else None

        # Drop rows with stock details (keep only rows with 'TOTAL' in Unnamed: 3)
        df = df[df[stock_details_col] == 'TOTAL']

        #drop rows where department details
        df = df[(df[state_col] != 'M/O Defence') & (df[state_col] != 'M/O Railways')]

        # Select only the relevant columns
        columns_to_keep = [state_col, med_code_col, med_desc_col] + [get_total_column(month, df.columns) for month in months]
        if total_col:
            columns_to_keep.append(total_col)
        df = df[columns_to_keep]

        # Rename columns
        df = df.rename(columns={
            state_col: 'state',
            med_code_col: 'medical_code',
            med_desc_col: 'medical_description'
        })

        # Rename the month columns
        for month in months:
            df = df.rename(columns={get_total_column(month, df.columns): month.lower()})
        if total_col:
            df = df.rename(columns={total_col: 'total'})

        # Add a 'Year' column
        df['year'] = f"{year}-{year+1}"

        # Rename Column names to lower case
        df.columns = df.columns.str.lower()

        # Remove rows where 'state' is in the state column
        df = df[df['state'].str.lower() != 'state']

        # Remove quotes from the start and end of medical_code
        df['medical_code'] = df['medical_code'].str.strip("'")

        # Append the dataframe to the list
        all_data.append(df)
    else:
        print(f"File not found for year {year}-{year+1}")

# Concatenate all dataframes in the list
combined_data = pd.concat(all_data, ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv("combined_all_india.csv", index=False)

print("Data consolidation complete. Output saved as 'combined_all_india.csv'")
print(f"Total rows in the combined data: {len(combined_data)}")

Data consolidation complete. Output saved as 'combined_all_india.csv'
Total rows in the combined data: 35208


In [4]:
newdata = pd.read_csv("/content/combined_all_india.csv")
newdata

,state,medical_code,medical_description,april,may,june,july,august,september,october,november,december,january,february,march,total,year
0,A & N Islands,1.1,Total number of pregnant women registered for ANC,432.0,406.0,400.0,498.0,436.0,445.0,429.0,458.0,371.0,425.0,488.0,422.0,5210.0,2017-2018
1,A & N Islands,1.1.1,"Out of the total ANC registered, number regist...",351.0,300.0,291.0,381.0,326.0,343.0,331.0,344.0,286.0,315.0,329.0,316.0,3913.0,2017-2018
2,A & N Islands,1.2.1,Number of PW given TT1,347.0,317.0,293.0,373.0,332.0,338.0,378.0,352.0,297.0,326.0,369.0,318.0,4040.0,2017-2018
3,A & N Islands,1.2.2,Number of PW given TT2,275.0,300.0,272.0,260.0,323.0,288.0,274.0,307.0,291.0,291.0,283.0,305.0,3469.0,2017-2018
4,A & N Islands,1.2.3,Number of PW given TT Booster,81.0,63.0,72.0,88.0,68.0,71.0,80.0,82.0,60.0,71.0,74.0,60.0,870.0,2017-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,West Bengal,25.2.1,Number of sick infants provided - Free Medicin...,7427.0,9829.0,7957.0,6479.0,6887.0,6719.0,7584.0,7623.0,7203.0,5212.0,4308.0,4141.0,81369.0,2019-2020
35204,West Bengal,25.2.2,Number of sick infants provided - Free Diagnos...,7935.0,8451.0,6772.0,5537.0,5858.0,5606.0,6677.0,7034.0,4934.0,4839.0,3928.0,3907.0,71478.0,2019-2020
35205,West Bengal,25.2.3,Number of sick infants provided - Free Home to...,2881.0,3203.0,2856.0,2842.0,2311.0,3416.0,3434.0,3133.0,2685.0,2043.0,2310.0,2292.0,33406.0,2019-2020
35206,West Bengal,25.2.4,Number of sick infants provided - Interfacilit...,1008.0,1065.0,894.0,802.0,1011.0,1238.0,1300.0,1398.0,1095.0,708.0,823.0,884.0,12226.0,2019-2020


In [5]:
newdata.to_csv("after2017.csv")
from google.colab import files
files.download("after2017.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>